#Predição do Bitcoin com RNN

###Esse projeto tem como objetivo construir um modelo que prevê os valores futuros de Bitcoin do dataset abaixo:
https://www.kaggle.com/mczielinski/bitcoin-historical-data

###Esse dataset contêm os valores de transações de Bitcoin de minuto a minuto desde 01/01/2012 até 22/04/2020, com as seguintes colunas:
- Timestamp (começo da janela de tempo de um minuto em horário Unix)
- Open (preço inicial)
- High (maior preço)
- Low (menor preço)
- Close (preço final)
- Volume_(BTC) (quantidade de Bitcoin transacionado)
- Volume_(Currency) (quantidade de moeda transacionado)
- Weighted_Price (preço médio)

Usaremos a variavel Timestamp para sequênciar os valores e tentaremos predizer a variável Weighted_Price.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
caminho_drive = '/content/drive/My Drive/colab/RNN/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Primeiro vamos importar as bibliotecas que usaremos durante todo o notebook.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

Agora importamos o arquivo para começar a análise.

In [0]:
nm_arquivo = 'bitstampUSD_1-min_data_2012-01-01_to_2020-04-22.csv'
arquivo = pd.read_csv( caminho_drive + nm_arquivo )

In [0]:
arquivo.describe()

In [0]:
arquivo.head()

In [0]:
x_plot = pd.to_datetime(arquivo['Timestamp'], unit='s')

plt.figure( figsize=(20, 8) )
plt.plot( x_plot, arquivo['Weighted_Price'] )

Primeira coisa que podemos perceber é que antes de 2017 temos muitos valores NaN, que significa que não tinhamos muitas transações da moeda. Vamos então filtrar esses dados com intuito de trabalharmos com os dados mais voláteis.

In [0]:
dt = datetime(2019, 1, 1)
timestamp = dt.replace(tzinfo=timezone.utc).timestamp()

arquivo_maior_2017 = arquivo[ arquivo['Timestamp'] >= timestamp ]

In [0]:
x_plot = pd.to_datetime(arquivo_maior_2017['Timestamp'], unit='s')

plt.figure( figsize=(20, 8) )
plt.plot( x_plot, arquivo_maior_2017['Weighted_Price'] )

In [0]:
arquivo_maior_2017.isna().sum()

Mesmo tirando o ano de 2017, ainda temos muitos casos NaN. Uma linha de Nan significa que não teve nenhuma transação na janela de tempo indicada, ou seja, o valor continuou igual, logo, uma ideia para preencher os valores vazios é de preencher todos os valores com o Close da última linha.


In [0]:
arquivo_maior_2017.sort_values( by=['Timestamp'] ) #garantido a seqência dos valores
arquivo_na = arquivo_maior_2017.copy()

for i in range( arquivo_maior_2017.shape[0] ):
    if (i % 100000 == 0):
        print( '{}/{} dados processados'.format( i, arquivo_maior_2017.shape[0] ) )
    if pd.isna( arquivo_maior_2017.iloc[i][1] ):
        valor_close = arquivo_na.iloc[i - 1][4]
        arquivo_na.iloc[i] = [ arquivo_na.iloc[i][0] , valor_close , valor_close , valor_close , valor_close , 0 , 0 , valor_close ]

In [0]:
arquivo_na.isna().sum()

Olhando a página do dataset o autor diz que podem ter timestamps que não existem, que no momento da janela de tempo, a API que captura os arquivos não estava funcionando. Vamos ver se é muito grande essa quantidade.

In [0]:
valor_minimo = arquivo_na['Timestamp'].min()
valor_maximo = arquivo_na['Timestamp'].max()

quantidade_desejada = int( ( valor_maximo - valor_minimo ) / 60 ) + 1
quantidade_real = arquivo_na.shape[0]

diferenca = quantidade_desejada - quantidade_real
diference_percentual = (diferenca / quantidade_desejada) * 100

print('São {0} linhas não existentes em {1} linhas'.format(diferenca, quantidade_desejada))

Como podemos ver, depois de 2017 não temos casos que a API não estava funcionando, então não precisamos nos preocupar.

Vamos agora decidir nossas variaveis da RNN.

Como ainda temos muito dados, podemos usar um numero de time steps grande (tomando cuidado para não deixar grande demais a ponto de estourar a RAM do Colab) e vamos utilizar todos os dados do arquivo, com objetivo de tentar uma boa predição.

As transações de Bitcoin são feitas com uma demora de em média 10 minutos, com isso, devemos tentar predizer valores com pelo menos 10 minutos a frente, ou seja, o nosso future_steps deve ser >= 10.

In [0]:
time_steps = 60
nr_features = arquivo_na.shape[1] - 1 #tiramos a variável de tempo
future_steps = 10

Agora podemos preparar os dados que vão entrar no modelo.

Primeiro vamos normalizar os dados, deixando todos eles com valores entre 0 e 1.

In [0]:
scaler = MinMaxScaler()

arquivo_np = arquivo_na[  arquivo_na.columns[1:] ].to_numpy() #pega todas as colunas, menos a de data, e transforma em uma array

dados_norm = scaler.fit_transform( arquivo_np )

dados_norm

Agora vamos coloca-los no formato certo para entrar no modelo, onde cada exemplo de X terá (time_steps) linhas e (nr_features) colunas.

In [0]:
qnt_total = len(dados_norm)

X = []
Y = []

for i in range (time_steps, qnt_total - future_steps):
    X.append(dados_norm[i - time_steps : i])
    Y.append(dados_norm[i : i + future_steps , -1])

X_array = np.array( X )
Y_array = np.array( Y )

print('Dimensão de X = {}'.format( X_array.shape ))
print('Dimensão de Y = {}'.format( Y_array.shape ))

In [0]:
X[0]

In [0]:
Y[0]

Agora vamos dividir os dados em treino, validação e teste.

In [0]:
porc_treino = 0.7
porc_validacao = 0.15
porc_teste = 0.15

qnt_treino = int( qnt_total * porc_treino )
qnt_validacao = int( qnt_total * porc_validacao )
qnt_teste = int( qnt_total * porc_teste )

X_treino = X_array[ : qnt_treino ]
X_validacao = X_array[ qnt_treino : qnt_treino + qnt_validacao ]
X_teste = X_array[ qnt_treino + qnt_validacao : ]

Y_treino = Y_array[ : qnt_treino ]
Y_validacao = Y_array[ qnt_treino : qnt_treino + qnt_validacao ]
Y_teste = Y_array[ qnt_treino + qnt_validacao : ]

print('{} dados no total'.format( qnt_total ))
print('{} dados para o treino'.format( X_treino.shape[0] ))
print('{} dados para a validação'.format( X_validacao.shape[0] ))
print('{} dados para o teste'.format( X_teste.shape[0] ))

Podemos agora ir para o nosso modelo.

In [0]:
model = Sequential()

model.add( LSTM(units=30, input_shape=(time_steps, nr_features)) )
#model.add( LSTM(units=30, input_shape=(time_steps, nr_features), return_sequences=True) )

#model.add( LSTM(units=20, return_sequences=True) )

#model.add( LSTM(units=10) )

model.add( Dense(units=future_steps) )

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy','mse'])

Com os dados preparados e o modelo criado, podemos agora começar o treino.

In [0]:
historico = model.fit(X_treino, Y_treino, epochs=100, batch_size=time_steps, validation_data=(X_validacao, Y_validacao), shuffle=False)

In [0]:
plt.plot(historico.history['loss'], label='Training')
plt.plot(historico.history['val_loss'], label='Validation')
plt.legend()